In [99]:
import os
import cv2
import json,pickle,math
import logging, argparse
import numpy as np
import pandas as pd
import yaml , argparse
import multiprocessing.dummy as mp
from deploy_utils import make_sure_path_exists
from deploy_utils import visualize_output_dict
from deploy_utils import Infer_From_Model
from deploy_utils import load_json
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [100]:
def postprocess(detection_boxes, detection_classes, detection_scores,detection_features,detection_threshold):
    """
    Function Definition : This function does default post processing for Extract Features script [ Infer model ] does Class wise Threshold
    """
    filtered_boxes, filtered_classes, filtered_scores,filtered_features = [], [], [],[]
    for b, c, s ,f in zip(detection_boxes, detection_classes, detection_scores,detection_features):
        if s>=(detection_threshold[c]):
            filtered_boxes += [b]
            filtered_classes += [c]
            filtered_scores += [s]
            filtered_features += [f]
    filtered_boxes = np.asarray(filtered_boxes)
    filtered_classes = np.asarray(filtered_classes)
    filtered_scores = np.asarray(filtered_scores)
    filtered_feature_array = np.asarray(filtered_features)
    return filtered_boxes, filtered_classes, filtered_scores,filtered_feature_array

In [101]:
input_images_dir = "/jidoka/workspace/jdisk2/Shahabas/241-03-2023 Length mis match/"
filename_type = ["23975695_V_0_OD.png"]
output_json_dir = "/jidoka/workspace/jdisk2/Shahabas/sidemismatch/sim_1/json"
os.makedirs(output_json_dir,exist_ok=True)
output_result_dir = "/jidoka/workspace/jdisk2/Shahabas/sidemismatch/sim_1/result"
os.makedirs(output_result_dir,exist_ok=True)
model_path = "/jidoka/simulations/jdisk2/jt.cust.Sansera_CR/sim_object_detection/Station_5_Side_Camera/sim_5/analysis/infer_model_4000/"
classnames =    ["None","Component_ID","Crack","Damage","Steel_Shot","Underfill","Scales","Flash","Embossing","Part_Without_Piercing","Bend","Rust","Scale_Pit_Mark",
                "Length_Mismatch","Side_Mismatch","corner_detection","Defect_16","Defect_17","Defect_18","Defect_19","Defect_20"]
detection_threshold = ["None",0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.1,0.3,0.3,0.3,0.3,0.3,0.3]

In [102]:
defectwise_split = "/jidoka/workspace/jdisk2/Shahabas/sidemismatch/sim_1/defectwise/"
for class_id in classnames[1:]:
    defectwise_path = os.path.join(defectwise_split,class_id)
    os.makedirs(defectwise_path,exist_ok=True)
    
ok_dir = defectwise_split+"/ok"
ng_dir = defectwise_split+"/ng"
os.makedirs(ok_dir,exist_ok=True)
os.makedirs(ng_dir,exist_ok=True)

In [103]:
def seperate_filename(input_images_dir):
    """
    FUnction Definition : this function extracts the filename which we require
    """
    print('----------------------------------------------------------------------------------------------------------')
    print("Total Number of Images in Directory :- ",len(os.listdir(input_images_dir)))
    print('----------------------------------------------------------------------------------------------------------')
    filenames = os.listdir(input_images_dir)
    total_count = []
    seperated_filenames = []
    i = 0
    for filename_id in range(len(filename_type)):
        for filename_cnt in filenames:
            if filename_type[filename_id] in filename_cnt:
                i += 1
                seperated_filenames += [filename_cnt]
            total_count = i 
        print("Total Number of Images Selected     :- ",total_count)
        print('----------------------------------------------------------------------------------------------------------')
    return seperated_filenames

In [104]:
seperated_filenames = seperate_filename(input_images_dir)
img_list = []
for file in seperated_filenames:
    list_path = os.path.join(input_images_dir,file)
    img_list.append(list_path)

----------------------------------------------------------------------------------------------------------
Total Number of Images in Directory :-  130
----------------------------------------------------------------------------------------------------------
Total Number of Images Selected     :-  9
----------------------------------------------------------------------------------------------------------


In [105]:
img_list

['/jidoka/workspace/jdisk2/Shahabas/241-03-2023 Length mis match/230324-151803_SMCR1002FO01_L_1_C_8_S_5_Cam_23975695_V_0_OD.png',
 '/jidoka/workspace/jdisk2/Shahabas/241-03-2023 Length mis match/230324-151716_SMCR1002FO01_L_1_C_1_S_5_Cam_23975695_V_0_OD.png',
 '/jidoka/workspace/jdisk2/Shahabas/241-03-2023 Length mis match/230324-151747_SMCR1002FO01_L_1_C_6_S_5_Cam_23975695_V_0_OD.png',
 '/jidoka/workspace/jdisk2/Shahabas/241-03-2023 Length mis match/230324-151759_SMCR1002FO01_L_1_C_7_S_5_Cam_23975695_V_0_OD.png',
 '/jidoka/workspace/jdisk2/Shahabas/241-03-2023 Length mis match/230324-151739_SMCR1002FO01_L_1_C_5_S_5_Cam_23975695_V_0_OD.png',
 '/jidoka/workspace/jdisk2/Shahabas/241-03-2023 Length mis match/230324-151724_SMCR1002FO01_L_1_C_2_S_5_Cam_23975695_V_0_OD.png',
 '/jidoka/workspace/jdisk2/Shahabas/241-03-2023 Length mis match/230324-151730_SMCR1002FO01_L_1_C_3_S_5_Cam_23975695_V_0_OD.png',
 '/jidoka/workspace/jdisk2/Shahabas/241-03-2023 Length mis match/230324-151834_SMCR1002FO0

In [106]:
# Numpy encoder for inference
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self,obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [107]:
# Save Json function to save json files from model
def save_json(output_dict,png_filename,output_pickle_dir):
    output_str = json.dumps(output_dict, cls=NumpyEncoder)
    if '.' in png_filename:
        filename_without_ext = os.path.splitext(png_filename)[0]
    else:
        filename_without_ext = png_filename
    json_save_path = os.path.join(output_pickle_dir, filename_without_ext+'.json')

    with open(json_save_path, 'w') as f:
        f.write(output_str)

In [108]:
selected_camera = [".png"]
def infer_tfinfer_and_save(model, selected_camera,img_path):
    json_save_dict = {}
    newoutput_dict = {}
    input_img_dir, img_name = os.path.split(img_path)
    if ".png" in img_name:
        img = cv2.imread(img_path)
        print("-----INFERENCE COMPLETED FOR :",img_name,"-----")
        for i in range(len(selected_camera)):
            if selected_camera[i] in img_name:

                output_dict = model.run_inference_for_single_image_features(img)
                
                detection_boxes = output_dict['detection_boxes']
                detection_classes = output_dict['detection_classes']           
                detection_scores = output_dict['detection_scores']
                detection_features = output_dict['detection_features'] 
                
                json_save_dict["detection_boxes"]=detection_boxes  
                json_save_dict["detection_classes"]=detection_classes 
                json_save_dict["detection_scores"]=detection_scores
                               
                 
                save_json(json_save_dict,img_name,output_json_dir)
                
                detection_boxes, detection_classes, detection_scores ,detection_features= postprocess(detection_boxes, detection_classes, detection_scores,detection_features,detection_threshold)    

                newoutput_dict["detection_scores"]=detection_scores
                newoutput_dict["detection_classes"]=detection_classes                
                newoutput_dict["detection_boxes"]=detection_boxes     
                newoutput_dict["detection_features"]=detection_features
                newoutput_dict["num_detections"]=len(detection_classes)

                category_index = {}
                for i, classname in enumerate(classnames):
                    category_index[i] = {'name': classname}  
                    
                save_json(newoutput_dict,img_name,output_result_dir)            


multiprocess = 2 
tf_infer_pool = []

model = Infer_From_Model(model_path, log_level=logging.DEBUG)

tf_infer_pool = []
for k in range(multiprocess):
    tf_infer_pool.append(mp.Pool(1))
    
images_for_tf_infer_pool = {}
for k in range(multiprocess):
    images_for_tf_infer_pool[k] = []

target_container = 0
for img_path in img_list:
    input_img_dir, img_name = os.path.split(img_path)
    images_for_tf_infer_pool[target_container].append(img_path)
    target_container += 1 
    if target_container == 2:
        target_container = 0

result_objs = []
for k in range(multiprocess):
    for j in range(len(images_for_tf_infer_pool[k])):
        result_objs.append(tf_infer_pool[k].apply_async(infer_tfinfer_and_save,(model,selected_camera,images_for_tf_infer_pool[k][j])))  
r = [result.get() for result in result_objs]


-----INFERENCE COMPLETED FOR : 230324-151803_SMCR1002FO01_L_1_C_8_S_5_Cam_23975695_V_0_OD.png -----
-----INFERENCE COMPLETED FOR : 230324-151716_SMCR1002FO01_L_1_C_1_S_5_Cam_23975695_V_0_OD.png -----
-----INFERENCE COMPLETED FOR : 230324-151759_SMCR1002FO01_L_1_C_7_S_5_Cam_23975695_V_0_OD.png -----
-----INFERENCE COMPLETED FOR : 230324-151724_SMCR1002FO01_L_1_C_2_S_5_Cam_23975695_V_0_OD.png -----
-----INFERENCE COMPLETED FOR : 230324-151834_SMCR1002FO01_L_1_C_9_S_5_Cam_23975695_V_0_OD.png -----
-----INFERENCE COMPLETED FOR : 230324-151747_SMCR1002FO01_L_1_C_6_S_5_Cam_23975695_V_0_OD.png -----
-----INFERENCE COMPLETED FOR : 230324-151739_SMCR1002FO01_L_1_C_5_S_5_Cam_23975695_V_0_OD.png -----
-----INFERENCE COMPLETED FOR : 230324-151730_SMCR1002FO01_L_1_C_3_S_5_Cam_23975695_V_0_OD.png -----
-----INFERENCE COMPLETED FOR : 230324-151735_SMCR1002FO01_L_1_C_4_S_5_Cam_23975695_V_0_OD.png -----


# Secound Time Run

In [109]:
classnames =    ["None","Component_ID","Crack","Damage","Steel_Shot","Underfill","Scales","Flash","Embossing","Part_Without_Piercing","Bend","Rust","Scale_Pit_Mark",
                "Length_Mismatch","Side_Mismatch","corner_detection","Defect_16","Defect_17","Defect_18","Defect_19","Defect_20"]
detection_threshold = ["None",0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.1,0.3,0.3,0.3,0.3,0.3]

output_result_dir = "/jidoka/workspace/jdisk2/Shahabas/sidemismatch/sim_1/rerun/"
os.makedirs(output_result_dir,exist_ok=True)

In [110]:
defectwise_split = "/jidoka/workspace/jdisk2/Shahabas/sidemismatch/sim_1/defectwise/"
for class_id in classnames[1:]:
    defectwise_path = os.path.join(defectwise_split,class_id)
    os.makedirs(defectwise_path,exist_ok=True)
    
ok_dir = defectwise_split+"/ok"
ng_dir = defectwise_split+"/ng"
os.makedirs(ok_dir,exist_ok=True)
os.makedirs(ng_dir,exist_ok=True)

In [111]:
def postprocess(detection_boxes, detection_classes, detection_scores,detection_threshold):
    """
    Function Definition : This function does default post processing for Extract Features script [ Infer model ] does Class wise Threshold
    """
    filtered_boxes, filtered_classes, filtered_scores = [], [], []
    for b, c, s  in zip(detection_boxes, detection_classes, detection_scores):
        if s>=(detection_threshold[c]):
            filtered_boxes += [b]
            filtered_classes += [c]
            filtered_scores += [s]
    filtered_boxes = np.asarray(filtered_boxes)
    filtered_classes = np.asarray(filtered_classes)
    filtered_scores = np.asarray(filtered_scores)
    return filtered_boxes, filtered_classes, filtered_scores

def get_concat_images(img1, img2, individual_shape=(1024,1024)):
    img1 = cv2.resize(img1, individual_shape)
    img2 = cv2.resize(img2, individual_shape)
    return np.concatenate([img1, img2], axis=1)

In [112]:
inference_images_dir = "/jidoka/workspace/jdisk2/Shahabas/sidemismatch/sim_1/inference"
os.makedirs(inference_images_dir,exist_ok= True)
json_filenames = os.listdir(output_json_dir)

for json_filename in json_filenames:
    newoutput_dict = {}
    image_id = json_filename[:-5]

    curr_img_name = image_id + ".png"
    print("Infering  ",curr_img_name)

    curr_img_path = os.path.join(input_images_dir, curr_img_name)

    
    curr_bbspecs = load_json(output_json_dir, json_filename)
    
    detection_classes = curr_bbspecs['detection_classes']           
    detection_boxes = curr_bbspecs['detection_boxes']
    detection_scores = curr_bbspecs['detection_scores']

    
    detection_boxes, detection_classes, detection_scores = postprocess(detection_boxes, detection_classes, detection_scores,detection_threshold)    

    newoutput_dict["detection_scores"]=detection_scores
    newoutput_dict["detection_classes"]=detection_classes                
    newoutput_dict["detection_boxes"]=detection_boxes     
    newoutput_dict["num_detections"]=len(detection_classes)

    category_index = {}
    for i, classname in enumerate(classnames):
        category_index[i] = {'name': classname}  

    save_json(newoutput_dict,curr_img_name,output_result_dir)

    curr_img = cv2.imread(curr_img_path)
    curr_img_vis = visualize_output_dict(curr_img, newoutput_dict,classnames=classnames[1:], min_score_thresh=0.0)   
    save_img_path = os.path.join(inference_images_dir, curr_img_name)
    images_side_by_side = get_concat_images(curr_img, curr_img_vis)
    cv2.imwrite(save_img_path, images_side_by_side)
    
    if len(newoutput_dict["detection_classes"]) == 0:
        defectwise_save_path = os.path.join(defectwise_split, "ok",curr_img_name)
        cv2.imwrite(defectwise_save_path, curr_img_vis)
        
    if len(newoutput_dict["detection_classes"]) != 0:
        defectwise_save_path = os.path.join(defectwise_split, "ng",curr_img_name)
        cv2.imwrite(defectwise_save_path, curr_img_vis)
        
    for class_id in newoutput_dict["detection_classes"]:
        defectwise_save_path = os.path.join(defectwise_split, classnames[class_id],curr_img_name)
        cv2.imwrite(defectwise_save_path, curr_img_vis)

Infering   230324-151747_SMCR1002FO01_L_1_C_6_S_5_Cam_23975695_V_0_OD.png
Infering   230324-151739_SMCR1002FO01_L_1_C_5_S_5_Cam_23975695_V_0_OD.png
Infering   230324-151724_SMCR1002FO01_L_1_C_2_S_5_Cam_23975695_V_0_OD.png
Infering   230324-151735_SMCR1002FO01_L_1_C_4_S_5_Cam_23975695_V_0_OD.png
Infering   230324-151834_SMCR1002FO01_L_1_C_9_S_5_Cam_23975695_V_0_OD.png
Infering   230324-151759_SMCR1002FO01_L_1_C_7_S_5_Cam_23975695_V_0_OD.png
Infering   230324-151716_SMCR1002FO01_L_1_C_1_S_5_Cam_23975695_V_0_OD.png
Infering   230324-151803_SMCR1002FO01_L_1_C_8_S_5_Cam_23975695_V_0_OD.png
Infering   230324-151730_SMCR1002FO01_L_1_C_3_S_5_Cam_23975695_V_0_OD.png
